# k-NN model

Given a playlist, we want to add more tracks: it's the **playlist continuation** problem. Following [Kelen et al.](https://dl.acm.org/doi/abs/10.1145/3267471.3267477), the idea here is to define a similarity metric between two playlists, select the $k$ most similar playlists to ours, define a score metric for tracks continuing our playlist and choose the best tracks to continue it.

In [1]:
from scipy.sparse import lil_matrix
from tqdm.notebook import tqdm
import glob
import numpy as np
import pandas as pd

from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

## Treatment

### Tracks

Here we load and treat the tracks dataset.

In [2]:
def tracks_dfs():
    for file in glob.glob('../../data/sp_tracks_ready_*.pkl'):
        df = pd.read_pickle(file)[['id', 'playlist_id', 'artists_ids']]
        yield pd.concat([df, pd.DataFrame({'file': [file]*len(df)})], axis=1)

In [3]:
tracks_df = pd.concat(tqdm(tracks_dfs(), total=128), ignore_index=True)
tracks_df.dropna(inplace=True)

# The following is necessary to discard repeated playlists
tracks_df['idx'] = tracks_df.index
grouped = tracks_df.groupby(['playlist_id', 'file'])['idx'].apply(list).reset_index()
tracks_df = tracks_df.drop(index = [el for list in grouped[grouped.duplicated('playlist_id')].idx for el in list])
del grouped
tracks_df.drop(columns='idx', inplace=True)

We treat the playlists dataset:

In [4]:
playlists = tracks_df.groupby('playlist_id')['id'].apply(list)

We treat the artists for each track:

In [5]:
artists = tracks_df.drop_duplicates('id').set_index('id').artists_ids
artists.index.name = 'track_id'
artists_ids = dict(zip(artists.index, range(len(artists))))

## Training and test data

Now we split training and test data:

In [6]:
# Only playlists between 5 and 250 tracks
query = playlists.apply(lambda x: len(x))
playlists = playlists[(query >= 5) & (query <= 250)]

In [7]:
# Split training and test data
n_test = int(np.ceil(len(playlists)*0.2))
query = playlists.apply(lambda x: len(x))
playlists_test = playlists[(query > 25)].sample(n_test)

In [8]:
playlists_training = playlists.drop(index = playlists_test.index)

## The model

### Relevance matrix $R$

We build the relevance matrix $R$.

$R_{ij}=r_{ij}$ indicates if a track $j$ is relevant to the playlist $i$, that is, the track is in the playlist.

In [9]:
all_tracks = []
for playlist in tqdm(playlists_training.to_list()):
    all_tracks.extend(playlist)
all_tracks = list(set(all_tracks))

Because we will use matrix multiplication, we have to index each track and each playlist id to index of the matrix. We do it here using dictionaries:

In [10]:
track_ids_go = dict(zip(all_tracks, range(len(all_tracks))))
track_ids_back = dict(zip(track_ids_go.values(), track_ids_go.keys()))
playlist_ids = dict(zip(set(playlists_training.index), range(len(set(playlists_training.index)))))

In [11]:
m = len(set(playlists_training.index))
n = len(set(all_tracks))
R = lil_matrix((m, n))

for playlist_id, playlist in tqdm(playlists_training.iteritems(), total=len(playlists_training)):
    for track_id in playlist:
        R[playlist_ids[playlist_id], track_ids_go[track_id]] = 1

### Similarity

The similarity between two playlists $u$ and $v$ is calculated by:
$$s_{uv} = \sum_{i \in I} \dfrac{r_{ui}r_{vi}}{||R_u||_2||R_v||_2}$$
$I$ is the set of tracks and $R_u$ is the vector of relevances $r_{ui}$ for the playlist $u$.

In fact, we basically count the number of tracks in the intersection of the playlists and normalize it.

In [12]:
def similarity(playlist_1, playlist_2):
    """Calculate the similarity between two playlists."""
    summation = len(set(playlist_1) & set(playlist_2))
    return summation/(np.sqrt(len(playlist_1))*np.sqrt(len(playlist_2)))

### Track score

Given a playlist $u$ to be continuated, we calculate the similarity of it with all existent playlists and select the $k$ most similar playlists, that is, the set $N_k(u)$. So, we define a score for a track to be in the playlist:
$$\hat{r}_{ui} = \dfrac{\sum_{v \in N_k(u)} s_{uv} \cdot r_{vi}}{\sum_{v \in N_k(u)} s_{uv}}$$

The intuition is that we are giving high scores to tracks that are in many playlists with great similarities to out playlist. We return the tracks ordered by score.

In [13]:
def continuation(playlist, k):
    """Continue a playlist based on k most similar playlists."""
    s_u = lil_matrix((1, m))
    for alt_playlist_index, alt_playlist in playlists_training.items():
        s_u[0, playlist_ids[alt_playlist_index]] = similarity(playlist, alt_playlist)
    sorted_similarities_indices = np.flip(np.argsort(s_u.toarray()[0]))
    top_k_similarities_indices = sorted_similarities_indices[:k]
    scores = (s_u[0, top_k_similarities_indices]*R[top_k_similarities_indices, :]).toarray()[0]
    sorted_scores_indices = np.flip(np.argsort(scores)[-500:])
    return [track_ids_back[index] for index in sorted_scores_indices]

## Evaluation

### R-precision metric

As described in their work, [Chen et al.](https://dl.acm.org/doi/10.1145/3240323.3240342) suggest a metric for playlist continuation evaluation. They call it **R-precision**. It measures how many of the real tracks (and their artists) the model suggested correctly.

A playlist as input to the model has two parts: its part on display to the model and it's hidden part. The hidden part is what the model try to predict and is called *ground truth*.

$$\textrm{R-precision} = \dfrac{|S_T \cap G_T| + 0.25 \cdot |S_A \cap G_A|}{|G_T|}$$

$G_T$ is the set of unique track IDs from ground truth, that is, the unique hidden tracks. $S_T$ is the suggested tracks from our model. $G_A$ is the set of unique artists IDs from ground truth and $S_A$ is the set of predicted artists. The metric can be interpreted as accuracy (although it can be greater than 1), but giving some score for wrong tracks with right artists.

In [14]:
def r_precision(S_t, G_t, S_a, G_a):
    return (len(set(S_t) & set(G_t)) + 0.25 * len(set(S_a) & set(G_a))) / len(G_t)

In [15]:
def evaluation(playlist_not_hidden, playlist_hidden, continuation):
    continuation = continuation.copy()
    for track in set(playlist_not_hidden):
        if track in continuation:
            continuation.remove(track)
    continuation = continuation[:len(playlist_hidden)]
    
    G_a = []
    for track in playlist_hidden:
        G_a.extend(artists.iloc[artists_ids[track]])
    S_a = []
    for track in continuation:
        S_a.extend(artists.iloc[artists_ids[track]])
        
    metric = r_precision(continuation, playlist_hidden, S_a, G_a)
    return metric

We now select a $k$ value that maximizes the R-precision metric in our test dataset.

In [18]:
for k in tqdm([1, 5, 10, 1000, 4000, 10000, len(playlists_test)]):
    sum_metric = 0
    # sum_random = 0
    for playlist in tqdm(playlists_test.sample(200)):
        playlist_not_hidden = playlist[:25]
        playlist_hidden = playlist[25:]
        continuated = continuation(playlist_not_hidden, k)
        metric = evaluation(playlist_not_hidden, playlist_hidden, continuated)
    #     random_metric = evaluation(playlist_not_hidden, playlist_hidden, random.sample(all_tracks, 500))
        sum_metric += metric
    #     sum_random += random_metric
    print('With k={}, the R-precision for our model is about {}.'.format(k, sum_metric/200))
    # print('A random model performs {}.'.format(sum_random/len(playlists_test)))


With k=1, the R-precision for our model is about 0.059119349930770566.



With k=5, the R-precision for our model is about 0.08112396859029525.



With k=10, the R-precision for our model is about 0.09701821526056069.



With k=1000, the R-precision for our model is about 0.10973446710248691.



With k=4000, the R-precision for our model is about 0.09078662201016692.



With k=10000, the R-precision for our model is about 0.10893415539611212.



With k=15241, the R-precision for our model is about 0.08637187650537527.



As said by [Chen et al.](https://dl.acm.org/doi/10.1145/3240323.3240342), the highest performance achieved in *RecSys Challenge 2018* was 0.2241. Well, many competitors were using much more advanced models, like neural networks, and they also had much more data and possible more computational power. So the results we achieved seems much reasonable.

## Sanity check

Before evaluating out model, it's worth it to do a sanity check. We will take the most listened songs from The Beatles and continue the playlist, with $k = 100$.

In [19]:
q = sp.search('michael jackson', type='artist')

Our playlist to be continuated:

In [20]:
[track['name'] for track in sp.artist_top_tracks(q['artists']['items'][0]['id'])['tracks']]

['Billie Jean',
 'Beat It - Single Version',
 'Smooth Criminal - 2012 Remaster',
 "Don't Stop 'Til You Get Enough - Single Version",
 'Don’t Matter To Me (with Michael Jackson)',
 'The Way You Make Me Feel - 2012 Remaster',
 'Rock with You - Single Version',
 "They Don't Care About Us",
 'P.Y.T. (Pretty Young Thing)',
 'Remember the Time']

Continuating...

In [21]:
temp = sp.artist_top_tracks(q['artists']['items'][0]['id'])['tracks']
result = continuation([track['id'] for track in temp], 1000)

So:

In [22]:
q = sp.tracks(result[:50])

In [23]:
[(q['tracks'][i]['name'], q['tracks'][i]['artists'][0]['name']) for i in range(len(q['tracks']))][:25]

[('Billie Jean', 'Michael Jackson'),
 ('Beat It - Single Version', 'Michael Jackson'),
 ("Don't Stop 'Til You Get Enough - Single Version", 'Michael Jackson'),
 ('P.Y.T. (Pretty Young Thing)', 'Michael Jackson'),
 ('Rock with You - Single Version', 'Michael Jackson'),
 ('Smooth Criminal - 2012 Remaster', 'Michael Jackson'),
 ('The Way You Make Me Feel - 2012 Remaster', 'Michael Jackson'),
 ('Remember the Time', 'Michael Jackson'),
 ('Bad - 2012 Remaster', 'Michael Jackson'),
 ('Take on Me', 'a-ha'),
 ('Thriller', 'Michael Jackson'),
 ("They Don't Care About Us", 'Michael Jackson'),
 ('Sweet Dreams (Are Made of This) - Remastered', 'Eurythmics'),
 ('Human Nature', 'Michael Jackson'),
 ('Man in the Mirror - 2012 Remaster', 'Michael Jackson'),
 ('Black or White', 'Michael Jackson'),
 ("Wanna Be Startin' Somethin'", 'Michael Jackson'),
 ('I Wanna Dance with Somebody (Who Loves Me)', 'Whitney Houston'),
 ('The Girl Is Mine', 'Michael Jackson'),
 ('Off the Wall', 'Michael Jackson'),
 ('Like 

Well, it seems nice.